In [ ]:
from imports import *
from ChannelShape import *
%matplotlib inline

In [ ]:
# Define channel shapes for CDMSlite detector
litechan = DetectorShape("CDMSlite1")
litechan.Add(ChannelShape("PA",1,31.3,36.1,-pi,pi,+1.,0., 0., 29.60, 34.75))
litechan.Add(ChannelShape("PB",1,0.,31.3,radians(90.),radians(210.),+1., 0., 0., 0., 29.60))
litechan.Add(ChannelShape("PC",1,0.,31.3,radians(210.),radians(330.),+1., 0., 0., 0., 29.60))
litechan.Add(ChannelShape("PD",1,0.,31.3,radians(-30.),radians(90.),+1., 0., 0., 0., 29.60))
litechan.Add(ChannelShape("Det",None,0.,38.1,-pi,pi,+1., 0., 75.4888/2., 0., 72.1868/2.).addFlat(37.7444,radians(45.)))
# NOTE: In CDMSZipInfo, X and Y flats are given by diameter, not radius

In [ ]:
print(litechan["Det"])

In [ ]:
# Draw detector outline, including flats
def drawDet(detShape):
    phi = np.arange(1000)*2.*pi/1000.
    edge = [detShape.getPointAtPhi(f) for f in phi]
    rlim = 1.1*detShape["Det"].rmax
    plt.figure(figsize=(6,6))    # Units are inches
    plt.plot(*zip(*edge))
    plt.axis('square')

In [ ]:
# Overlay circle at point on existing plot
def addCircle(center, radius):
    phi = np.arange(1000)*2.*pi/1000.
    circ = np.array([ChannelShape.pol2cart((radius,f)) for f in phi])
    circ += center
    plt.plot(*zip(*circ))
    plt.plot(*center,'*')

In [ ]:
drawDet(litechan)
addCircle((15,15), 25)

In [ ]:
# Need some basic 2D vector operations
def vlen(vec):
    return sqrt(vec[0]**2+vec[1]**2)

def vdot(v1,v2):
    return (v1[0]*v2[0]+v1[1]*v2[1])

def vsum(v1,v2):
    return (v1[0]+v2[0], v1[1]+v2[1])

def vdiff(v1,v2):
    return (v1[0]-v2[0], v1[1]-v2[1])

def vmult(vec, val):
    return (val*vec[0],val*vec[1])
    
def reflectPoint(pos, center, detShape):
    if (detShape.contains(pos)): return pos     # No action if inside
        
    fromPos = vdiff(center, pos)
    edge = detShape.getPointOnEdge(center, fromPos)   # Intersection
    norm = detShape.getOutwardNormal(edge)            # Unit vector
    fromEdge = vdiff(pos, edge)
    reflEdge = vsum(fromEdge, vmult(norm, -2*vdot(fromEdge,norm)))

    return vsum(edge, reflEdge)

In [ ]:
# Need some basic 2D vector operations
def vlen(vec):
    return sqrt(vec[0]**2+vec[1]**2)

def vdot(v1,v2):
    return (v1[0]*v2[0]+v1[1]*v2[1])

def vsum(v1,v2):
    return (v1[0]+v2[0], v1[1]+v2[1])

def vdiff(v1,v2):
    return (v1[0]-v2[0], v1[1]-v2[1])

def vmult(vec, val):
    return (val*vec[0],val*vec[1])
    
def reflectPoint(pos, center, detShape):
    if (detShape.contains(pos)): return pos     # No action if inside
        
    fromPos = vdiff(center, pos)
    edge = detShape.getPointOnEdge(center, fromPos)   # Intersection
    norm = detShape.getOutwardNormal(edge)            # Unit vector
    fromEdge = vdiff(pos, edge)
    reflEdge = vsum(fromEdge, vmult(norm, -2*vdot(fromEdge,norm)))

    return vsum(edge, reflEdge)

In [ ]:
def foldedCircle(detShape, center, radius):
    phi = np.arange(1000)*2.*pi/1000.
    
    # Do this old school with a loop, then make it better
    fold = []
    for f in phi:
        cpos = vsum(ChannelShape.pol2cart((radius,f)), center)
        refl = reflectPoint(cpos, center, detShape)
        fold.append(refl)

    return fold

In [ ]:
hit = (15,15)
dist = 25

drawDet(litechan)
addCircle(hit, dist)
fold = foldedCircle(litechan, hit, dist)
plt.plot(*zip(*fold))
plt.savefig("CDMSlite_folded-circle.png")

In [ ]:
def genGauss(x, scale=1, alpha=1, beta=2, mu=0):
    return np.exp(-(abs(x-mu)/alpha)**beta)

def genGauss2D(pos, **kwargs):
    """Pass position as (x,y), additional for genGauss as:
       alpha = width of distribution
       beta = exponent for power expression
       mu = offset of "radius" in power expression
       x0, y0 = coordinates of offset center (mu = sqrt(x0^2+y0^2))
       center = coordinates of offset as tuple (x0,y0)
    """
    # If center point specified, subtract it off, don't use mu
    if "x0" in kwargs and "y0" in kwargs:
        kwargs["mu"] = 0.
    elif "center" in kwargs:
        x0,y0, = center
        kwargs["mu"] = 0.
    else:
        x0,y0 = (0.,0.)

    r = np.sqrt((pos[0]-x0)**2+(pos[1]-y0)**2)
    return genGauss(r, kwargs)
    
def foldedGauss(detShape, center, radius, **kwargs):
    """Pass detector outline, center and radius of full disk.
       Additional arguments alpha, beta are for the generalized Gaussian."""

    # Create a mesh in polar coordinates around center
    r = np.linspace(0., radius, 50)
    p = np.linspace(0., 2*np.pi, 1000)
    R,P = np.meshgrid(r,p)
    Z = genGauss(R, alpha=alpha, beta=beta)    # Simpler than genGauss2D 

    # Convert the mesh to Cartesian with offset for drawing
    X,Y = R*np.cos(P)+center[0], R*np.sin(P)+center[1]

    # Process all the Cartesian points through the folding
    foldX,foldY = np.zeros_like(X),np.zeros_like(Y)
    for i in np.ndindex(X.shape):
        foldX[i],foldY[i] = reflectPoint((X[i],Y[i]),center,detShape)

    return foldX,foldY,Z

In [ ]:
hit = (15,15)
dist = 25

alpha=3.5
beta=0.8
gx,gy,gz = foldedGauss(litechan, hit, dist, alpha=alpha, beta=beta)

drawDet(litechan)
addCircle((15,15), 25)
plt.scatter(gx,gy,c=gz, cmap=plt.cm.plasma, alpha=0.05, s=9)
plt.savefig("CDMSlite_folded-gaussian.png")